크롤러 테스트용 20개 검색용 크롤러입니다.
- 현재는 20개의 리스트에서 이름으로 검색하며 크롤링하는 방식
- 실제 진행시에는 게임 목록에서 각 게임별 링크를 받아와 링크로 각 게임 페이지로 이동하며 크롤링 진행할 예정

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

import time
import platform
import requests
from bs4 import BeautifulSoup

In [ ]:
# MonogoDB연결 정보
import pymongo

USER = "USER"
PWD = "PWD"
HOST = "HOST"
PORT = "PORT"
client = pymongo.MongoClient(f"mongodb://{USER}:{PWD}@{HOST}:{PORT}")

In [ ]:
# db 이름 지정
db = client['DB_name']
db

In [ ]:
# 맥이면 이렇게
if platform.system() == 'Darwin':
    def get_chrome_driver():
        driver = webdriver.Chrome(
            executable_path="/opt/homebrew/bin/chromedriver"
            )
    
        return driver

# 윈도우면 이렇게
elif platform.system() == 'Windows':
    def get_chrome_driver():
        # 1. 크롬 옵션 세팅
        chrome_options = webdriver.ChromeOptions()

        # 2. driver 생성하기
        driver = webdriver.Chrome(
            service=Service(ChromeDriverManager().install()),
           options=chrome_options
        )
    
        return driver
else:
    print('Unknown System... sorry~~~~')

In [ ]:
driver = get_chrome_driver()

In [ ]:
# 테스트용 리스트
lst = ["DON'T STARVE TOGETHER", "Rust", "THE SIMS4", "CUPHEAD", "MUSEDASH",
"GEOMETRY DASH", "DNF DUEL", "Rusty Lake: Roots", "Cyberpunk 2077",
"FarCry 6", "RIMWORLD", "OXYGEN NOT INCLUDED", "DOKI DOKI LITERATURE CLUB!",
"PERSONA 4 GOLDEN", "DEAD BY DAYLIGHT", "STARDEW VALLEY", "THE FOREST",
"CRUSADER KINGS III", "GRAND THEFT AUTO V"]

In [ ]:
for i in range(len(lst)):
    
    id = int(i) + 1
    
    # 수정될 부분(게임 링크 리스트로 들어갈 예정)
    game_URL = f"https://store.steampowered.com/search/?term={lst[i]}"
    
    driver.get(game_URL)
    time.sleep(1)
    
    # 검색어 리스트에서 첫번째 페이지 불러오기
    link = driver.find_element(By.CSS_SELECTOR, "#search_resultsRows")
    soup = BeautifulSoup(link.get_attribute("innerHTML"), "html.parser")
    a = soup.select_one("a")['href']
    driver.get(a)
    time.sleep(1)
    
    try:
        detail = driver.find_element(By.CSS_SELECTOR,"#genresAndManufacturer").text
    except:
        # 연령 입력 상황이 발생하는 경우
        driver.find_element(By.ID, "ageYear").send_keys("1990")
        driver.find_element(By.ID, "view_product_page_btn").click()
        time.sleep(2)
        detail= driver.find_element(By.CSS_SELECTOR,"#genresAndManufacturer").text
    
    # title
    title = detail.split('\n')[0].split(":")[1:]
    title = title[0].strip()

    # genre
    genre = detail.split('\n')[1].split(":")[1:]
    genre = genre[0].strip()

    # developer
    developer = detail.split('\n')[2].split(":")[1:]
    developer = developer[0].strip()

    # publisher
    publisher = detail.split('\n')[3].split(":")[1:]
    publisher = publisher[0].strip()

    # franchise가 없는 경우가 있어서 release_date와 합쳐서 처리
    temp_franchise = detail.split('\n')[4].split(":")[1:]
    temp_var1 = detail.split('\n')[4].split(":")[0]

    if temp_var1 == "FRANCHISE":
        franchise = temp_franchise[0].strip()
        # release_date
        release_date = detail.split('\n')[5].split(":")[1:]
        release_date = release_date[0].strip()
    elif temp_var1 == "RELEASE DATE":
        franchise = "0"
        release_date = temp_franchise[0].strip()
    
    time.sleep(2)
    
    # recent_reviews 평가
    recent_reviews = driver.find_element(By.CSS_SELECTOR, "#userReviews > div:nth-child(1) > div.summary.column > span").text
    
    recent_reviews_temp = driver.find_element(By.CSS_SELECTOR, '#userReviews > div:nth-child(1) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc')
    # recent_reviews 평가 비율(최근 30일)
    recent_reviews_ratio = recent_reviews_temp.get_attribute("innerHTML").strip().split(" ")[1].replace("%","")
    # recent_reviews 점수를 준 유저 수(최근 30일)
    recent_reviews_voted_users = recent_reviews_temp.get_attribute("innerHTML").strip().split(" ")[4].replace(",","")

    # 전체기간 유저 평가 신작 게임은 없는 경우가 있다.
    try:
        all_reviews = driver.find_element(By.CSS_SELECTOR, "#userReviews > div:nth-child(2) > div.summary.column > span.game_review_summary.positive").text
    
        all_reviews_temp = driver.find_element(By.CSS_SELECTOR, '#userReviews > div:nth-child(2) > div.summary.column > span.nonresponsive_hidden.responsive_reviewdesc')
        # 전체기간 유저 평가 비율
        all_reviews_ratio = all_reviews_temp.get_attribute("innerHTML").strip().split(" ")[1].replace("%","")
        # 전체기간 유저 점수를 준 유저 수
        all_reviews_voted_users = all_reviews_temp.get_attribute("innerHTML").strip().split(" ")[4].replace(",","")
    except:
        all_reviews = "0"
        all_reviews_ratio = "0"
        all_reviews_voted_users = "0"
    
    # image_link
    image_link = driver.find_element(By.CSS_SELECTOR, "img.game_header_image_full").get_attribute("src")

    # grade
    try:
        grade = driver.find_element(By.CSS_SELECTOR, "div.game_rating_icon > img").get_attribute("src").split("/")[-1].split(".")[0]
    except:
        grade = "0"
    time.sleep(2)
    # info
    info_lst = []
    info_group = driver.find_element(By.CSS_SELECTOR, "div.game_area_features_list_ctn")
    info_soup = BeautifulSoup(info_group.get_attribute("innerHTML"), "html.parser")
    parsed_info = info_soup.select("div.label")
    for i in parsed_info:
        info_lst.append(i.text)

    # tags
    tag_group = driver.find_element(By.CSS_SELECTOR, "div.glance_tags")
    tag_soup = BeautifulSoup(tag_group.get_attribute("innerHTML"), "html.parser")
    parsed_tag = tag_soup.select("a")
    tag_lst =[]
    for i in parsed_tag:
        tag_lst.append(i.text.strip())
    
    # about_this_game
    about_this_game = driver.find_element(By.CSS_SELECTOR, "#game_area_description").text.replace("\n\n"," ").replace("\n", " ")[16:]

    time.sleep(2)
    # language info
    language_group = driver.find_element(By.CSS_SELECTOR, "table.game_language_options > tbody")
    language_soup = BeautifulSoup(language_group.get_attribute("innerHTML"), "html.parser")
    parsed_language = language_soup.select("tr")
    parsed_language = parsed_language[1:]

    # language를 리스트에 담아두기
    language_list = []
    for i in parsed_language:
        language_list.append(i.select_one("td.ellipsis").text.strip())
    
    # language_list 길이만큼 0을 요소로 갖는 리스트 생성
    interface_list = [0 for _ in range(len(language_list))]

    # len(language_list) 만큼 반복하며 해당 언어가 interface 항목을 만족하는지 체크
    for i in range(len(language_list)):
        try:
            if parsed_language[i].select_one("td.checkcol > span").text == '✔':
                # interface_list에 해당 언어 입력
                interface_list[i] = language_list[i]
        except:
            continue

    fullaudio_list = [0 for _ in range(len(language_list))]
    for i in range(len(language_list)):
        try:
            if parsed_language[i].select_one("#languageTable > table > tbody > tr > td:nth-child(3)").text == '✔':
                fullaudio_list[i] = language_list[i]
        except:
            continue

    subtitles_list = [0 for _ in range(len(language_list))]
    for i in range(len(language_list)):
        try:
            if parsed_language[i].select_one("#languageTable > table > tbody > tr > td:nth-child(4)").text == '✔':
                subtitles_list[i] = language_list[i]
        except:
            continue
    
    DOC = {"id":id, "title":title, "genre":genre, "developer":developer, "publisher":publisher,
           "franchise":franchise, "release_date":release_date, "recent_reviews":recent_reviews,
           "recent_reviews_ratio":int(recent_reviews_ratio), "recent_reviews_voted_users":int(recent_reviews_voted_users),
           "all_reviews":all_reviews,
           "all_reviews_ratio":int(all_reviews_ratio), "all_reviews_voted_users":int(all_reviews_voted_users),
           "image_link":image_link, "grade":grade, "info":info_lst, "tag":tag_lst, "about_this_game":about_this_game,
           "language_interface":interface_list, "language_fullaudio":fullaudio_list, "language_subtitles":subtitles_list}
    db.test.insert_one(DOC)
    time.sleep(4)
    del info_lst, tag_lst, interface_list, fullaudio_list, subtitles_list